In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import  precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,auc, roc_curve
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

import mlflow
from mlflow.models import infer_signature

import requests


In [60]:
ROOT_PATH = '../data/'
PATH = ROOT_PATH + 'lending_data.csv'
TARGET_COL = 'default.payment.next.month'

SEED = 42

In [61]:
df = pd.read_csv(PATH)

In [62]:
df = df.drop('ID', axis = 1)

In [63]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = SEED)

In [64]:
X_train = train_set.drop(['default.payment.next.month'], axis = 'columns')
y_train = train_set['default.payment.next.month']

X_test = test_set.drop(['default.payment.next.month'], axis = 1)
y_test = test_set['default.payment.next.month']

In [65]:
scaler = MinMaxScaler()

features_names = X_train.columns

X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = features_names)

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test, columns = features_names)

In [132]:
import json

with open('../config/app.json') as f:
    config = json.load(f)

In [133]:
MLFLOW_TRACKING_URI = f"http://localhost:{config['tracking_port']}"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [134]:
model_name = "random_forest"
model_version = "champion"
f"models:/{model_name}@{model_version}"


'models:/random_forest@champion'

In [135]:
model = mlflow.pyfunc.load_model(f"models:/{model_name}@{model_version}")
model

mlflow.pyfunc.loaded_model:
  artifact_path: rf_pipeline
  flavor: mlflow.sklearn
  run_id: 5ed50ee89d31494898a204a0e91cc95c

In [137]:
X_test

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,0.020202,0.0,0.333333,0.666667,0.068966,0.2,0.2,0.2,0.2,0.2,...,0.092703,0.171988,0.094248,0.191233,0.001717,0.001187,0.001674,0.002415,0.003517,0.003794
1,0.141414,1.0,0.166667,0.666667,0.086207,0.2,0.2,0.2,0.2,0.2,...,0.150409,0.255826,0.157733,0.244564,0.005135,0.002514,0.003528,0.004262,0.006257,0.005063
2,0.060606,1.0,0.500000,0.333333,0.189655,0.2,0.2,0.2,0.2,0.2,...,0.123970,0.225844,0.150168,0.238541,0.002783,0.001848,0.003348,0.003926,0.005861,0.004845
3,0.121212,0.0,0.500000,0.666667,0.482759,0.2,0.2,0.2,0.2,0.2,...,0.095224,0.176055,0.091789,0.184498,0.001843,0.001073,0.007828,0.000043,0.016437,0.008362
4,0.040404,1.0,0.333333,0.666667,0.258621,0.2,0.2,0.2,0.2,0.2,...,0.109603,0.178576,0.100772,0.195171,0.002290,0.000891,0.001116,0.002899,0.000000,0.001897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.353535,1.0,0.166667,0.666667,0.103448,0.3,0.0,0.1,0.1,0.1,...,0.086800,0.160138,0.081908,0.178720,0.000000,0.000493,0.000000,0.002047,0.000420,0.003737
5996,0.494949,1.0,0.166667,0.666667,0.120690,0.4,0.2,0.2,0.4,0.2,...,0.147667,0.233679,0.158363,0.247680,0.006869,0.007808,0.000000,0.004831,0.011723,0.018970
5997,0.050505,1.0,0.166667,0.333333,0.120690,0.2,0.2,0.2,0.2,0.2,...,0.113081,0.207600,0.132254,0.223549,0.002404,0.001484,0.002790,0.004026,0.003751,0.002846
5998,0.010101,1.0,0.166667,0.666667,0.137931,0.3,0.1,0.1,0.1,0.1,...,0.088523,0.161423,0.081243,0.178567,0.002855,0.002355,0.001529,0.000966,0.000000,0.000000


In [136]:
y_preds = model.predict(X_test)
y_preds

MlflowException: Failed to enforce schema of data '      LIMIT_BAL  SEX  EDUCATION  MARRIAGE       AGE  PAY_0  PAY_2  PAY_3  \
0      0.020202  0.0   0.333333  0.666667  0.068966    0.2    0.2    0.2   
1      0.141414  1.0   0.166667  0.666667  0.086207    0.2    0.2    0.2   
2      0.060606  1.0   0.500000  0.333333  0.189655    0.2    0.2    0.2   
3      0.121212  0.0   0.500000  0.666667  0.482759    0.2    0.2    0.2   
4      0.040404  1.0   0.333333  0.666667  0.258621    0.2    0.2    0.2   
...         ...  ...        ...       ...       ...    ...    ...    ...   
5995   0.353535  1.0   0.166667  0.666667  0.103448    0.3    0.0    0.1   
5996   0.494949  1.0   0.166667  0.666667  0.120690    0.4    0.2    0.2   
5997   0.050505  1.0   0.166667  0.333333  0.120690    0.2    0.2    0.2   
5998   0.010101  1.0   0.166667  0.666667  0.137931    0.3    0.1    0.1   
5999   0.505051  1.0   0.500000  0.333333  0.689655    0.2    0.2    0.2   

      PAY_4  PAY_5  ...  BILL_AMT3  

In [94]:
sum(y_preds)

np.int64(689)

In [122]:
new_df = test_set
new_df['predictions'] = y_preds
new_df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month,predictions
2308,30000.0,1,2,2,25,0,0,0,0,0,...,13716.0,14828.0,1500.0,2000.0,1500.0,1500.0,1500.0,2000.0,0,0
22404,150000.0,2,1,2,26,0,0,0,0,0,...,77741.0,77264.0,4486.0,4235.0,3161.0,2647.0,2669.0,2669.0,0,0
23397,70000.0,2,3,1,32,0,0,0,0,0,...,70111.0,70212.0,2431.0,3112.0,3000.0,2438.0,2500.0,2554.0,0,0
25058,130000.0,1,3,2,49,0,0,0,0,0,...,11236.0,6944.0,1610.0,1808.0,7014.0,27.0,7011.0,4408.0,0,0
2664,50000.0,2,2,2,36,0,0,0,0,0,...,20295.0,19439.0,2000.0,1500.0,1000.0,1800.0,0.0,1000.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,360000.0,2,1,2,27,1,-2,-1,-1,-1,...,1271.0,179.0,0.0,830.0,0.0,1271.0,179.0,1970.0,0,0
14144,500000.0,2,1,2,28,2,0,0,2,0,...,78376.0,80912.0,6000.0,13151.0,0.0,3000.0,5000.0,10000.0,1,1
23108,60000.0,2,1,1,28,0,0,0,0,0,...,52045.0,52661.0,2100.0,2500.0,2500.0,2500.0,1600.0,1500.0,0,0
25703,20000.0,2,1,2,29,1,-1,-1,-1,-1,...,600.0,0.0,2494.0,3967.0,1370.0,600.0,0.0,0.0,0,0


In [130]:
new_df[(new_df['default.payment.next.month'] == new_df['predictions']) & (new_df['predictions'] == 1)].iloc[10].to_dict()

{'LIMIT_BAL': 220000.0,
 'SEX': 1.0,
 'EDUCATION': 1.0,
 'MARRIAGE': 2.0,
 'AGE': 29.0,
 'PAY_0': 1.0,
 'PAY_2': 2.0,
 'PAY_3': 2.0,
 'PAY_4': 2.0,
 'PAY_5': 2.0,
 'PAY_6': 2.0,
 'BILL_AMT1': 31012.0,
 'BILL_AMT2': 30215.0,
 'BILL_AMT3': 33117.0,
 'BILL_AMT4': 32286.0,
 'BILL_AMT5': 34320.0,
 'BILL_AMT6': 33634.0,
 'PAY_AMT1': 0.0,
 'PAY_AMT2': 3400.0,
 'PAY_AMT3': 0.0,
 'PAY_AMT4': 2576.0,
 'PAY_AMT5': 0.0,
 'PAY_AMT6': 2671.0,
 'default.payment.next.month': 1.0,
 'predictions': 1.0}